<a href="https://colab.research.google.com/github/lizhjort/Insight/blob/master/Merge_genre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import lightgbm as lgb

In [0]:
data_with_checkouts = pd.read_pickle('/content/gdrive/My Drive/insight_data/data_with_checkouts.pkl.gz')

In [0]:
checkouts = pd.read_pickle('/content/gdrive/My Drive/insight_data/checkouts.pkl.gz')

In [0]:
inventory = pd.read_pickle('/content/gdrive/My Drive/insight_data/min_inventory.pkl.gz')

In [0]:
checkouts.head()

,BibNumber,ItemBarcode,CallNumber,ItemTitle,CheckoutDateTime
0,2856320,10090971945,741.5973 En62C04 2012,Crossed Volume 4,2017-09-01 08:03:00
20,2802097,10076863678,FIC FOLLETT 2012,Winter of the world,2017-09-01 09:18:00
32,3259846,10091244367,417.209 D356V 2017,Vulgar tongues an alternative history of Engli...,2017-09-01 09:20:00
38,3141712,10086563177,126 B566C 2007,Conversations on consciousness what the best m...,2017-09-01 09:20:00
40,3246991,10090698043,500 D322S 2017,Science in the soul selected writings of a pas...,2017-09-01 09:20:00


In [0]:
genre = checkouts['CallNumber'].str.split(" ", n = 1, expand = True) 


In [0]:
genre.drop([1], axis=1, inplace=True)

In [0]:
genre = genre[0].str.split(".", n=1, expand=True)

In [0]:
genre.drop([1], axis=1, inplace=True)

In [0]:
def label_encode(series, min_count):
    vc = series.value_counts()
    le = {c:i for i, c in enumerate(vc.index[vc >= min_count])}
    return le
  
le = None
if le is None:
  le_genre = label_encode(genre[0], min_count=1000)
else:
  le_genre = le

In [0]:
checkouts['Genre'] = genre
checkouts['Genre'] = checkouts['Genre'].map(le_genre).fillna(-1).astype(int)

In [0]:
checkouts

,BibNumber,ItemBarcode,CallNumber,ItemTitle,CheckoutDateTime,Genre
0,2856320,10090971945,741.5973 En62C04 2012,Crossed Volume 4,2017-09-01 08:03:00,6
20,2802097,10076863678,FIC FOLLETT 2012,Winter of the world,2017-09-01 09:18:00,0
32,3259846,10091244367,417.209 D356V 2017,Vulgar tongues an alternative history of Engli...,2017-09-01 09:20:00,-1
38,3141712,10086563177,126 B566C 2007,Conversations on consciousness what the best m...,2017-09-01 09:20:00,-1
40,3246991,10090698043,500 D322S 2017,Science in the soul selected writings of a pas...,2017-09-01 09:20:00,144
55,67969,10002659414,917.9 D747M,My wilderness the Pacific West,2017-09-01 09:21:00,9
62,2371303,10052633939,FIC ZWEIG2006,Somewhere in Germany an autobiographical novel,2017-09-01 09:23:00,0
63,3078444,10085581386,940.356 R63F 2015,fall of the Ottomans the Great War in the Midd...,2017-09-01 09:23:00,29
64,3091988,10091181353,SCI-FIC BRADBUR 2011,Martian chronicles,2017-09-01 09:24:00,5
65,2047749,10083452200,SCI-FIC MCCAFFR,Dragonsdawn,2017-09-01 09:24:00,5


In [0]:
data_with_checkouts.head()

,BibNumber,CheckoutDate,CheckoutCount,CheckoutCount1WeekPrev,CheckoutCount2WeekPrev,CheckoutCountNextWeek
0,10143,2017-09-03,0.0,NaN,NaN,0.0
1,10143,2017-09-10,0.0,0.0,NaN,0.0
2,10143,2017-09-17,0.0,0.0,0.0,0.0
3,10143,2017-09-24,0.0,0.0,0.0,0.0
4,10143,2017-10-01,0.0,0.0,0.0,0.0


In [0]:
#checkouts['Genre'].value_counts()
unique_genre = checkouts['Genre'].unique()
unique_genre = pd.DataFrame(data=unique_genre)
unique_genre.head()

,0
0,741
1,FIC
2,417
3,126
4,500


In [0]:
checkouts = checkouts.drop(['ItemBarcode', 'CallNumber',	'ItemTitle',	'CheckoutDateTime'], axis=1)

In [0]:
checkouts.head()

,BibNumber,Genre
0,2856320,6
20,2802097,0
32,3259846,-1
38,3141712,-1
40,3246991,144


In [0]:
checkouts = checkouts.sort_values(by='BibNumber', axis=0)

In [0]:
checkouts = checkouts.drop_duplicates()


In [0]:
checkouts.shape

(247405, 2)

In [0]:
data_with_checkouts.head()

,BibNumber,CheckoutDate,CheckoutCount,CheckoutCount1WeekPrev,CheckoutCount2WeekPrev,CheckoutCountNextWeek
0,10143,2017-09-03,0.0,NaN,NaN,0.0
1,10143,2017-09-10,0.0,0.0,NaN,0.0
2,10143,2017-09-17,0.0,0.0,0.0,0.0
3,10143,2017-09-24,0.0,0.0,0.0,0.0
4,10143,2017-10-01,0.0,0.0,0.0,0.0


In [0]:
data = pd.merge(data_with_checkouts, checkouts, on=['BibNumber'], how='left')

In [0]:
data

,BibNumber,CheckoutDate,CheckoutCount,CheckoutCount1WeekPrev,CheckoutCount2WeekPrev,CheckoutCountNextWeek,Genre
0,10143,2017-09-03,0.0,NaN,NaN,0.0,72.0
1,10143,2017-09-10,0.0,0.0,NaN,0.0,72.0
2,10143,2017-09-17,0.0,0.0,0.0,0.0,72.0
3,10143,2017-09-24,0.0,0.0,0.0,0.0,72.0
4,10143,2017-10-01,0.0,0.0,0.0,0.0,72.0
5,10143,2017-10-08,0.0,0.0,0.0,0.0,72.0
6,10143,2017-10-15,0.0,0.0,0.0,0.0,72.0
7,10143,2017-10-22,0.0,0.0,0.0,0.0,72.0
8,10143,2017-10-29,0.0,0.0,0.0,0.0,72.0
9,10143,2017-11-05,0.0,0.0,0.0,0.0,72.0


In [0]:
data.to_pickle('/content/gdrive/My Drive/insight_data/data_7_col.pkl.gz')